In [ ]:
# Set Jupyter to render directly to the screen
%matplotlib inline

# Import pandas and numpy for analysis
import pandas as pd
import numpy as np
import matplotlib as plt
import math as math

# Importing the Black Scholes functions

In [ ]:
from black_scholes import call_value, put_value, call_delta, put_delta, call_vega, put_vega

In [ ]:
help(call_value)

In [ ]:
help(put_value)

In [ ]:
help(call_delta)

# Testing the Black-Scholes function

In [ ]:
# This is an example showing the functionality of the black_scholes module by making a plot
# of the option values and greeks for different prices of the underlying stock. You will need 
# to use the same functions, call_value, put_value, call_delta, etc. with different inputs 
# for your own trading strategy.

K = 100
T = 0.4
r = 0.0
sigma = 0.16

stock_values = range(80, 122, 2)

call_values = list()
put_values = list()
call_deltas = list()
put_deltas = list()
call_vegas = list()
put_vegas = list()

for stock_value in stock_values:
    call_values.append(call_value(stock_value, K, T, r, sigma))
    put_values.append(put_value(stock_value, K, T, r, sigma))
    call_deltas.append(call_delta(stock_value, K, T, r, sigma))
    put_deltas.append(put_delta(stock_value, K, T, r, sigma))
    call_vegas.append(call_vega(stock_value, K, T, r, sigma))
    put_vegas.append(put_vega(stock_value, K, T, r, sigma))

    
option_values = pd.DataFrame(data={'Call': call_values, 'Put': put_values}, index=stock_values)
option_values.index.name = 'Stock Value'

option_deltas = pd.DataFrame(data={'Call': call_deltas, 'Put': put_deltas}, index=stock_values)
option_deltas.index.name = 'Stock Value'

option_vegas = pd.DataFrame(data={'Call': call_vegas, 'Put': put_vegas}, index=stock_values)
option_vegas.index.name = 'Stock Value'

option_values.plot(title='Option Value - Strike {}'.format(K))
option_deltas.plot(title='Option Delta - Strike {}'.format(K))
option_vegas.plot(title='Option Vega - Strike {}'.format(K))

# Reading in the CSV file

In [ ]:
def read_data(filename):
    '''
    This function reads the .csv stored at the 'filename' location and returns three DataFrame's.
    
    The first DataFrame contains the Stock bid and ask price and volumes. It is indexed by timestamp.
    There is one level of column names, which describes whether it is a BidPrice, BidVolume, AskPrice
    or AskVolume.
    
    The second DataFrame contains the Time-to-Expiry of the options in the dataset. It is indexed by
    timestamp, and has one level of column names. There is only one column: the time to expiry.
  
    The third DataFrame contains the Option bid and ask price and volumes for different strikes. It
    is indexed by timestamp and has three levels of columns. The first level gives the strike, the second
    level gives the option type (C for Call and P for put), the third level gives whether it is a
    BidPrice, BidVolume, AskPrice or AskVolume.
    '''
    df = pd.read_csv(filename, index_col=0)
       
    time_to_expiry = df.filter(like='TimeToExpiry')
    
    stock = df.filter(like='Stock')
    stock.columns = [stock.columns.str[-5:], stock.columns.str[:-6]]
    
    options = pd.concat((df.filter(like='-P'), df.filter(like='-C')), axis=1)  
    options.columns = [options.columns.str[-3:], options.columns.str[:-4]]

    market_data = pd.concat((stock, options), axis=1)
    
    return time_to_expiry, market_data

In [ ]:
# Read the market data

filename = 'Options Arbitrage.csv'
time_to_expiry, market_data = read_data(filename)

In [ ]:
# Get a list of all instrument names including the stock, and of the options only
instrument_names = list(market_data.columns.get_level_values(0).unique())
print(instrument_names)

option_names = instrument_names[1:]
print(option_names)

# Some Examples

In [ ]:
# What is in the market_data dataframe? 
market_data.head()

In [ ]:
# What is in the time to expiry dataframe?
time_to_expiry.tail()

In [ ]:
# Plot bids and asks for the stock
market_data['Stock', 'BidPrice'].head(300).plot(figsize=(20, 3))
market_data['Stock', 'AskPrice'].head(300).plot(figsize=(20, 3))

In [ ]:
# Plot bids and asks for two of the options
print(option_names[1], option_names[4])

market_data[option_names[1], 'BidPrice'].head(300).plot(figsize=(20, 3))
market_data[option_names[1], 'AskPrice'].head(300).plot(figsize=(20, 3))
market_data[option_names[4], 'BidPrice'].head(300).plot(figsize=(20, 3))
market_data[option_names[4], 'AskPrice'].head(300).plot(figsize=(20, 3))

# Exercise

In [ ]:
# Now it's your turn. Start by calculating your own theoretical values and deltas for the options. Visualize 
# these alongside the market data, is it clear when you would want to do a trade? Enter positions when that 
# is the case, and manage your delta-risk by hedging away your delta position in the underlying stock.

In [ ]:
# Merge market_data with time to expiry and calculate mid prices

market_data["TimeToExpiry"] = time_to_expiry["TimeToExpiry"]

market_data["Stock", "MidPrice"] = (market_data["Stock", "AskPrice"] + market_data["Stock", "BidPrice"])/2

market_data.head()

In [ ]:
# Calculate theoretical prices and greeks for options using a for loop to make it easy
# if we buy a put, we also buy the underlying stock so we need the ask price of the stock to calcualate the value and delta.
# the same goes for when we sell a call. The opposite goes for when we buy a call or sell a put. Using the bid and ask prices
# as stock price in these calculations is easier than using the midprice, because that would complicate the algorithm later on

for option in option_names:
    if option[0] == "P":
        market_data[option, "ValueAsk"] = put_value(market_data["Stock", "AskPrice"],
                                           int(option[1:]),
                                           market_data["TimeToExpiry"],
                                           0,
                                           0.2)
        market_data[option, "ValueBid"] = put_value(market_data["Stock", "BidPrice"],
                                           int(option[1:]),
                                           market_data["TimeToExpiry"],
                                           0,
                                           0.2)
        market_data[option, "DeltaAsk"] = put_delta(market_data["Stock", "AskPrice"],
                                           int(option[1:]),
                                           market_data["TimeToExpiry"],
                                           0,
                                           0.2)
        market_data[option, "DeltaBid"] = put_delta(market_data["Stock", "BidPrice"],
                                           int(option[1:]),
                                           market_data["TimeToExpiry"],
                                           0,
                                           0.2)
        market_data[option, "DeltaMid"] = put_delta(market_data["Stock", "MidPrice"],
                                           int(option[1:]),
                                           market_data["TimeToExpiry"],
                                           0,
                                           0.2)
    if option[0] == "C":
        market_data[option, "ValueAsk"] = call_value(market_data["Stock", "BidPrice"],
                                           int(option[1:]),
                                           market_data["TimeToExpiry"],
                                           0,
                                           0.2)
        market_data[option, "ValueBid"] = call_value(market_data["Stock", "AskPrice"],
                                           int(option[1:]),
                                           market_data["TimeToExpiry"],
                                           0,
                                           0.2)
        market_data[option, "DeltaAsk"] = call_delta(market_data["Stock", "BidPrice"],
                                           int(option[1:]),
                                           market_data["TimeToExpiry"],
                                           0,
                                           0.2)
        market_data[option, "DeltaBid"] = call_delta(market_data["Stock", "AskPrice"],
                                           int(option[1:]),
                                           market_data["TimeToExpiry"],
                                           0,
                                           0.2)
        market_data[option, "DeltaMid"] = call_delta(market_data["Stock", "MidPrice"],
                                           int(option[1:]),
                                           market_data["TimeToExpiry"],
                                           0,
                                           0.2)

market_data = market_data.sort_index(axis = 1, ascending = False)
market_data.head()

In [ ]:
# Examine data when the theoretical ask value is higher than the ask price (undervalued call option)
market_data.loc[market_data["C60", "ValueAsk"] > market_data["C60", "AskPrice"]].head()

In [ ]:
# Visualizing the problem:
# when the stock price goes up, the value of a call option
# goes up and the reverse happens for put options

market_data["Stock", "BidPrice"].plot(figsize = (20,10))
market_data["P80", "ValueAsk"].plot(color = 'r', label = "P80",legend = True)
market_data["P60", "ValueAsk"].plot(color = 'y', label = "P60",legend = True)
market_data["P70", "ValueAsk"].plot(color = 'black', label = "P70",legend = True)
market_data["C80", "ValueAsk"].plot(color = 'g', label = "C80",legend = True)
market_data["C60", "ValueAsk"].plot(color = 'b', label = 'C60',legend = True)
market_data["C70", "ValueAsk"].plot(color = 'purple', label = "C70", legend = True)


In [ ]:
# Visualizing: how do the deltas of the options evolve
# call deltas move in the same way as the call values, but
# put deltas move in the opposite directino of the put values
# makes sense because delta is about change, but not the direction of the change

market_data["P60", "DeltaAsk"].plot(color = 'r', figsize = (20,10))
# market_data["P60", "PutValue"].plot(color = 'r')
market_data["P70", "DeltaAsk"].plot(color = 'b')
# market_data["P70", "PutValue"].plot(color = 'b')

market_data["C60", "DeltaAsk"].plot(color = 'g')
# market_data["C80", "CallValue"].plot(color = 'g')


In [ ]:
# For each option, calculate the difference between our theoretical ask value and the actual ask price, and
# the difference between our theoretical bid value and the actual bid price. If the ask price is lower than 
# the theoretical price, it is undervalued (so if AskDifference > 0). 
# If the bid price is higher than theoretical price, it is overvalued (so if BidDifference > 0)
for option in option_names:
    market_data[option, "AskDifference"] = (market_data[option, "ValueAsk"] - market_data[option, "AskPrice"])
    market_data[option, "BidDifference"] = (market_data[option, "BidPrice"] - market_data[option, "ValueBid"])

market_data = market_data.sort_index(axis = 1, ascending = False)

market_data.head()

In [ ]:
# For one option, make a graph with all the instances where the AskDifference > 0 (so undervalued option)
P80 = market_data.loc[market_data["P80", "AskDifference"] > 0]
P80["P80", "AskDifference"].plot()

In [ ]:
# Do the same for all options, graph with instances where the options are undervalued
for option in option_names:
    OptionAsk = market_data.loc[market_data[option, "AskDifference"] > 0]    
    OptionBid = market_data.loc[market_data[option, "BidDifference"] > 0]
    OptionAsk[option, "AskDifference"].plot()
    OptionBid[option, "BidDifference"].plot()

In [ ]:
# Calculate the average of the AskDifference of all options, which we will use as our trading offset.
# So how much the ask price is lower than the theoreticl ask value, on average

meandifference = []
for option in option_names:
    OptionAsk = market_data.loc[market_data[option, "AskDifference"] > 0]    
    OptionBid = market_data.loc[market_data[option, "BidDifference"] > 0]
    meandifference.append(OptionAsk[option, "AskDifference"].mean())
    meandifference.append(OptionBid[option, "BidDifference"].mean())
    
trading_offset = sum(meandifference) / len(meandifference)
trading_offset

In [ ]:
option_deltas = []
for option in option_names:
    option_deltas.append([option,"Delta"])
    
option_deltas

In [ ]:
values = [1,2,5,1,2,5,2]
result = 0
for value in values:
    result += value
print(result)

In [ ]:
## Create hedge_ratio function to determine ratio of options to underlying stock 

# Function to determine if number is whole
def is_whole(n):
    return n % 1 == 0

# List of numbers from 99 to 1 that 100 can be divided by to get a whole number: [50,25,20,10,5,4,2,1]
integer_numbers = []
for i in reversed(range(1,100)):
    if is_whole(100/i):
        integer_numbers.append(i)

# Check if the rounded delta can be divided by a number in the list integer_numbers so that it is a whole number
# for example: if delta is 0.701300, then the rounded delta is 70 and it can be divided by 10 -> hedge ratio = 10/7,
# since 100/10 is a whole number (10) and 70/10 is a whole number (7) -> trade 10 options and 7 underlying stocks
def hedge_ratio(delta, integer_numbers):
    for number in integer_numbers:
        if is_whole(round(abs(delta) * 100) / number):
            hedge_ratio.options_ratio = 100/number
            hedge_ratio.stocks_ratio = round(abs(delta) * 100) / number
            break

In [ ]:
hedge_ratio(-0.444272, integer_numbers)
print(hedge_ratio.options_ratio)
print(hedge_ratio.stocks_ratio)

In [ ]:
# Once we have the hedge_ratio, we want to know how many options and stocks we can trade

# For example, hedge_ratio is 4/3 (4 options, 3 stocks), option ask volume = 23 and stock bid bolume = 77
# -> buy 20 options, sell 15 stocks which is in the ratio 4/3 (and vice versa if we sell a call or buy a put, of course)
option_ratio = 4
stock_ratio = 3
option_volume = 23
stock_volume = 77

options_trade = option_volume-(option_volume%option_ratio)
stocks_trade = (options_trade/option_ratio)*stock_ratio

print(options_trade, stocks_trade)

# Put the above code into a function -> if the result = 0 (when the hedge_ratio is e.g. 100/91, we do not want to trade)
def trade_made(options_ratio, stocks_ratio, option_volume, stock_volume):
    trade_made.options_trade = option_volume-(option_volume%options_ratio)
    trade_made.stocks_trade = (trade_made.options_trade/options_ratio)*stocks_ratio

trade_made(25,11,117,147)
print(trade_made.options_trade, trade_made.stocks_trade)

In [ ]:
def total_delta(market_data, option_names, positions):
    total_delta.total_delta=0
    for option in option_names:
        delta = market_data[option, "DeltaMid"] * positions[option]
        total_delta.total_delta += delta
    total_delta.total_delta += positions["Stock"]
    


In [ ]:
y = 3
z = 4
x = 0

for i in range(0,4):
    x += i
x += -10

if x >= 4:
    y += -round(x)
    z = x - round(x)
elif x <= -4:
    y += -round(x)
    z = x -round(x)
    
print(x,y,z)   

In [ ]:
#     if total_delta >= 19:
#         positions.loc[time, "Stock"] += -round(total_delta)
# #         positions.loc[time, 'TotalDelta'] = total_delta - 19
# #         positions.loc[time, 'TotalDelta'] = total_delta -round(total_delta)
#     elif total_delta <= -19:
#         positions.loc[time, 'Stock'] += round(total_delta)
# #         positions.loc[time, 'TotalDelta'] = total_delta + 19
# #         positions.loc[time, 'TotalDelta'] += -round(total_delta)

In [ ]:
### Algorithm 

# Create positions dataframe with stock and options
positions = pd.DataFrame(data=0, 
                         index=market_data.index,
                         columns= option_names)
positions["Stock"] = 0
positions["TotalDelta"] = 0
positions = positions.sort_index(axis = 1, ascending = False)

# Max absolute delta is 20
delta_limit = 20   
# Loop over the timestamps
prev_time = None
for time, mkt_data_at_time in market_data.iterrows():
    if prev_time == None:
        # This skips the first observation, we don't want to take a position yet
        prev_time = time
        continue
    # Loop over the put and call options at each timestamp
    temporary_stock=0
    temporary_stock2=0
    
    for option in option_names:
        
        # Calculate gamma and change in delta
#         gamma = mkt_data_at_time[]

    # If the ask price of an option is lower than the theoretical price by more than our required offset,
    # we want to buy the option
        if mkt_data_at_time[option, "AskDifference"] > trading_offset:

         # First specify the trades for call options
            if option[0] == 'C':
                
            # Get option volume and stock volume. Buy the option so ask volume, and sell the stock so bid volume.
                option_volume = mkt_data_at_time[option, "AskVolume"]
                stock_volume = mkt_data_at_time["Stock", "BidVolume"]

                # Use delta to determine hedge ratio, how much of the option compared to underlying stock is needed
                hedge_ratio(mkt_data_at_time[option, "DeltaAsk"], integer_numbers)

                # Use the results from the hedge_ratio function to determine how much we are going to trade, by
                # taking into account the available option and stock volume
                trade_made(hedge_ratio.options_ratio, hedge_ratio.stocks_ratio, option_volume, stock_volume)

                # Buy the call so +, and sell the stock so -
                positions.loc[time, option] = positions.loc[prev_time, option] + trade_made.options_trade
#                 positions.loc[time, "Stock"] += -trade_made.stocks_trade
                temporary_stock+= -trade_made.stocks_trade
                
#                 print(-trade_made.stocks_trade)
#                 print(positions.loc[time, "Stock"])
#                 print(temporary_stock)
         
            # Now for put options
            elif option[0] == 'P':
                
            # Get option volume and stock volume. Buy the option so ask volume, and buy the stock so ask volume.
                option_volume = mkt_data_at_time[option,"AskVolume"]
                stock_volume = mkt_data_at_time["Stock", "AskVolume"]

                # Use delta to determine hedge ratio, how much of the option compared to underlying stock is needed
                hedge_ratio(mkt_data_at_time[option, "DeltaAsk"], integer_numbers)

                # Use the results from the hedge_ratio function to determine how much we are going to trade, by
                # taking into account the available option and stock volume
                trade_made(hedge_ratio.options_ratio, hedge_ratio.stocks_ratio, option_volume, stock_volume)

                # Buy the put so +, and sell the stock so +
                positions.loc[time, option] = positions.loc[prev_time, option] + trade_made.options_trade
#                 positions.loc[time, "Stock"] += trade_made.stocks_trade
                temporary_stock2+= trade_made.stocks_trade

#                 print(trade_made.stocks_trade)
#                 print(positions.loc[time, "Stock"])
#                 print(temporary_stock2)



# #                 Now we do the same, but this time we sell the options (so look at BidDifference)
        elif mkt_data_at_time[option, "BidDifference"] > trading_offset:
                
            if option[0] == 'C':
                
            # Get option volume and stock volume. Sell the option so bid volume, and buy the stock so ask volume.
                option_volume = mkt_data_at_time[option, "BidVolume"]
                stock_volume = mkt_data_at_time["Stock", "AskVolume"]

                # Use delta to determine hedge ratio, how much of the option compared to underlying stock is needed
                hedge_ratio(mkt_data_at_time[option, "DeltaBid"], integer_numbers)

                # Use the results from the hedge_ratio function to determine how much we are going to trade, by
                # taking into account the available option and stock volume
                trade_made(hedge_ratio.options_ratio, hedge_ratio.stocks_ratio, option_volume, stock_volume)

                # Sell the call so -, and buy the stock so +
                positions.loc[time, option] = positions.loc[prev_time, option] - trade_made.options_trade
#                 positions.loc[time, "Stock"] += trade_made.stocks_trade
                temporary_stock+= trade_made.stocks_trade
                
#                 print(trade_made.stocks_trade)
#                 print(positions.loc[time, "Stock"])
#                 print(temporary_stock)

        
        # Now for put options
            elif option[0] == 'P':
                
            # Get option volume and stock volume. Sell the option so bid volume, and sell the stock so bid volume.
                option_volume = mkt_data_at_time[option,"BidVolume"]
                stock_volume = mkt_data_at_time["Stock", "BidVolume"]

                # Use delta to determine hedge ratio, how much of the option compared to underlying stock is needed
                hedge_ratio(mkt_data_at_time[option, "DeltaBid"], integer_numbers)

                # Use the results from the hedge_ratio function to determine how much we are going to trade, by
                # taking into account the available option and stock volume
                trade_made(hedge_ratio.options_ratio, hedge_ratio.stocks_ratio, option_volume, stock_volume)

                # Buy the put so +, and Buy the stock so +
                positions.loc[time, option] = positions.loc[prev_time, option] - trade_made.options_trade
#                 positions.loc[time, "Stock"] += -trade_made.stocks_trade
                temporary_stock2+= -trade_made.stocks_trade
                
#                 print(-trade_made.stocks_trade)
#                 print(positions.loc[time, "Stock"])
#                 print(temporary_stock2)


        else:
            positions.loc[time, option] = positions.loc[prev_time, option]
#             positions.loc[time, "Stock"] = positions.loc[prev_time, "Stock"]
            
#     if positions.loc[time, "Stock"] != positions.loc[prev_time, "Stock"]:
#         positions.loc[time, "Stock"] += positions.loc[prev_time, "Stock"]
    positions.loc[time, "Stock"] = positions.loc[prev_time, "Stock"] + temporary_stock2 + temporary_stock
    
    total_delta=0
    for option in option_names:
#         delta = mkt_data_at_time[option, "DeltaMid"] * positions.loc[time,option]
        total_delta += mkt_data_at_time[option, "DeltaMid"] * positions.loc[time,option]
    total_delta += positions.loc[time, "Stock"]
#     positions.loc[time, "TotalDelta"] = total_delta        

    if total_delta >= delta_limit:
        positions.loc[time, "Stock"] += -round(total_delta)
#         positions.loc[time, 'TotalDelta'] = total_delta - 19
        positions.loc[time, 'TotalDelta'] = total_delta -round(total_delta)
    elif total_delta <= -delta_limit:
        positions.loc[time, 'Stock'] += -round(total_delta)
#         positions.loc[time, 'TotalDelta'] = total_delta + 19
        positions.loc[time, 'TotalDelta'] = total_delta -round(total_delta)
    else:
        positions.loc[time, "TotalDelta"] = total_delta        
        
        # In the next iteration of the loop, the previous time will be what is now the current time
    prev_time = time

In [ ]:
# Visualize positions over time
positions.iloc[0:1000].plot(figsize=(20, 3))


In [ ]:
positions.tail()

In [ ]:
positions['TotalDelta'].plot(figsize=(20, 3))


In [ ]:
positions.loc[abs(positions['TotalDelta']) > 20]

In [ ]:
positions.loc["2018-01-08 06:15:00":"2018-01-08 06:30:00"]

In [ ]:
market_data.loc["2018-01-08 06:15:00":"2018-01-08 06:30:00"]

In [ ]:
# Traded lots, assume we trade 0 in the first period
trades = positions.diff().fillna(0)

# Again start with an empty DataFrame
pnl_trades = pd.DataFrame(index = trades.index, columns = list(positions))


# Calculate total PnL made from trading (we bought against ask price, and sold against bid price).
lots_bought = np.maximum(trades, 0)
lots_sold = -np.minimum(trades, 0)

for position in list(positions):
    if position[0] != 'T':
        pnl_trades[position] = lots_sold[position] * market_data[position, 'BidPrice'] - lots_bought[position] * market_data[position, 'AskPrice']

pnl_trades_total = pnl_trades.iloc[:,:].sum(axis=1)
pnl_trades_cumulative = pnl_trades_total.cumsum()

In [ ]:
# Evaluating the position at current midprice per stock
position_valuation = pd.DataFrame(data = 0, index = trades.index, columns = list(positions))

for position in list(positions):
    if position[0] != 'T':
        position_valuation[position] = (market_data[position, 'AskPrice'] + market_data[position, 'BidPrice'])/2 * positions[position]

position_valuation_total = position_valuation.iloc[:,:].sum(axis=1)

position_valuation_total.tail()

In [ ]:
# Total pnl from trades and position
pnl_total = pnl_trades_cumulative + position_valuation_total

# Plot it
pnl_trades_total.plot(figsize=(20, 3))

In [ ]:
# Visualize cumulative profit and loss
pnl_trades_cumulative.plot(figsize=(20, 3))

In [ ]:
# Visualize total pnl

pnl_total.plot(figsize=(20, 3), title='PnL for whole dataset')

In [ ]:
# Create output dataframe with positions and the PnL
output = pd.concat((positions, pnl_total), axis=1)
output.columns.values[[-1]] = ['Profit-and-Loss']
output.tail()

In [ ]:
# Create CSV output file 
output.to_csv("PnL.csv")